In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tqdm
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from torch.optim.lr_scheduler import StepLR
import os
import scipy.io as sio
import pickle

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
import random
random.seed(42)

---
## Misc

Functions for Evaluation

In [4]:
def pairwise_evaluation(vectors_real, vectors_new):
    count = 0
    total = 0
    vectors_real_cpu = vectors_real.detach().cpu().numpy()
    vectors_new_cpu = vectors_new.detach().cpu().numpy()

    for i in range(vectors_real_cpu.shape[0]):
        for j in range(vectors_real_cpu.shape[0]):
            if j > i:
                errivsi = np.corrcoef(vectors_new_cpu[i,:], vectors_real_cpu[i,:])
                errivsj = np.corrcoef(vectors_new_cpu[i,:], vectors_real_cpu[j,:])
                errjvsi = np.corrcoef(vectors_new_cpu[j,:], vectors_real_cpu[i,:])
                errjvsj = np.corrcoef(vectors_new_cpu[j,:], vectors_real_cpu[j,:])

                if (errivsi[0,1] + errjvsj[0,1]) > (errivsj[0,1] + errjvsi[0,1]):
                    count += 1
                total += 1

    accuracy = count / total
    return accuracy

In [5]:
def top_5(pred, real):
    counter = 0.0
    counterr = 0.0
    counter_ten= 0.0
    for i in range(pred.shape[0]):
        if np.argmax(pred[i,:])==np.argmax(real[i,:]):
            counter+=1
        sort = np.flip(np.argsort(pred[i,:]))
        holder = np.isin(np.argmax(real[i,:]),sort[:5])
        holder_ten = np.isin(np.argmax(real[i,:]),sort[:10])
        if holder:
            counterr+=1
        if holder_ten:
            counter_ten+=1
    accuracy = counter/pred.shape[0]
    accuracy_five = counterr/pred.shape[0]
    accuracy_ten = counter_ten/pred.shape[0]
    return accuracy, accuracy_five, accuracy_ten

Loss Functions

In [6]:
class CosineProximityLoss(nn.Module):
    def __init__(self):
        super(CosineProximityLoss, self).__init__()

    def forward(self, input1, input2):
        # Normalize the input tensors
        input1 = F.normalize(input1, p=2, dim=-1)
        input2 = F.normalize(input2, p=2, dim=-1)

        # Compute the cosine similarity
        cosine_sim = torch.sum(input1 * input2, dim=-1)

        # Cosine proximity loss is 1 - cosine similarity
        loss = 1 - cosine_sim.mean()

        return loss

In [7]:
class MeanDistanceLoss(nn.Module):
    def __init__(self, cosine_loss):
        super(MeanDistanceLoss, self).__init__()
        self.cosine_loss = cosine_loss
        self.val = 179

    def forward(self, y_true, y_pred):
        total = 0
        total_two = 0
        
        for i in range((self.val + 1)):
            if i == 0:
                total += (self.val * self.cosine_loss(y_true, y_pred))
            else:
                rolled = torch.roll(y_pred, i, dims=0)
                total_two -= self.cosine_loss(y_true, rolled)
        
        return (total_two / self.val) + (total / self.val)

---
## Data Loader functions 

In [8]:
def ROI_loader(subject, fil):
    data_path = './data/subjects/'
    all_data = sio.loadmat(data_path + subject + '/' + fil)      
    ROI = all_data['meta']
    Gordon_areas = ROI[0][0][11][0][14]   
    try:
        data = all_data['examples']
    except KeyError:
        data = all_data['examples_passagesentences']
    return Gordon_areas, data

In [9]:
def coltocoord_ROI_ordering(subject, fil):
    data_path = '../data/subjects/'
    all_data = sio.loadmat(data_path + subject + '/' + fil)      
    ROI = all_data['meta']
    coord = ROI[0][0][5]
    return coord

In [10]:
def matcher(area, last_dim, last, Roi_coord):
    small = np.sum(last[1])
    big = np.sum(last_dim[1])
    difference = big-small
    assert 0<=difference
    assert small == area.shape[1]
    if last_dim.shape[1]>last.shape[1]:
        helper = np.zeros_like(last_dim)
        helper[:,:last.shape[1]] = last
        last = helper
        checker = 0
    else:
        checker = last.shape[1] - last_dim.shape[1]
        helper = np.zeros_like(last)
        helper[:,checker:] = last_dim
        last_dim = helper

    area_new = np.zeros((area.shape[0],big))
    counter = 0
    holder = 0
    index = 0
    index_old = 0
    mean = np.zeros((area.shape[0],1))
    while counter<last_dim.shape[1]:
        if last_dim[1,counter] > (last[1,counter]+holder) and checker == 0:
            if last[1,counter] == 0:
                if last[1,counter-1] == 0:
                    area_new[:,index+last[1,counter]:index+last_dim[1,counter]-holder] = np.tile(mean,(1,last_dim[1,counter] - (last[1,counter]+holder)))
                    index+= (last_dim[1,counter]-holder)
                else:
                    mean = np.reshape(np.mean(area[:,(Roi_coord[:,2] == last[0,counter-1])], axis=1),(area.shape[0],1))
                    area_new[:,index+last[1,counter]:index+last_dim[1,counter]-holder] = np.tile(mean,(1,last_dim[1,counter] - (last[1,counter]+holder)))
                    index+= (last_dim[1,counter]-holder)
            else:
                mean = np.reshape(np.mean(area[:,(Roi_coord[:,2] == last[0,counter])], axis=1),(area.shape[0],1))
                area_new[:,index:index+last[1,counter]] = area[:,index_old:index_old+last[1,counter]]
                if difference > 0 and 0<=(difference - (last_dim[1,counter] - (last[1,counter]+holder))):
                    area_new[:,index+last[1,counter]:index+last_dim[1,counter]-holder] = np.tile(mean,(1,last_dim[1,counter] - (last[1,counter]+holder)))
                    difference = difference - (last_dim[1,counter] - (last[1,counter]+holder))
                    index+= (last_dim[1,counter] - holder)
                elif difference > 0 :
                    area_new[:,index+last[1,counter]:index+last[1,counter]+difference] = np.tile(mean,(1,difference))
                    index += (difference + last[1,counter])
                    difference = 0
                else:
                    index+= last[1,counter]
            holder = 0
        else:
            area_new[:,index:index+last[1,counter]] = area[:,index_old:index_old+last[1,counter]]
            index+= last[1,counter]
            if checker > 0:
                checker-=1
                holder = last[1,counter] + holder
            else:
                holder = (last[1,counter] + holder) - last_dim[1,counter]
        index_old+=last[1,counter]

        counter+=1
    return area_new

In [11]:
def class_sizer(subject):
    no_sent = ['M05','M06','M10', 'M13','M16', 'M17']
    only_two = ['M03']
    only_three = [ 'M08', 'M09','M14']
    both = ['P01','M02','M03','M04','M07','M15']
    if subject in no_sent:
        value = 4530
        value_test = 0
    if subject in only_two:
        value = 4287
        value_test = 243
    if subject in only_three:
        value = 4146
        value_test = 384
    if subject in both:
        value = 3903
        value_test = 627
    return value, value_test

In [12]:
def dataloader_sentence_word_split_new_matching_all_subjects(subject):
    data_path = '../data/subjects/'
    vector_path_180 = '../data/glove_data/180_concepts_real.mat'
    vector_path_243 = '../data/glove_data/243_sentences_real.mat'
    vector_path_384 = '../data/glove_data/384_sentences_real.mat'

    vector_180 = sio.loadmat(vector_path_180)['data']
    vector_243 = sio.loadmat(vector_path_243)['data']
    vector_384 = sio.loadmat(vector_path_384)['data']

    subjects = ['P01','M02','M03','M04','M05','M06','M07', 'M08', 'M09','M15','M10', 'M13','M14','M16', 'M17']
    no_sent = ['M05','M06','M10', 'M13','M16', 'M17']
    only_two = ['M03']
    only_three = [ 'M08', 'M09','M14']
    both = ['P01','M02','M03','M04','M07','M15']


    sizes = np.load('../data/look_ups/sizes.npy')
    last_dim_all = np.load('../data/look_ups/last_dim.npz')

    if subject in no_sent:
        value = 4530
        value_test = 0
    if subject in only_two:
        value = 4287
        value_test = 243
    if subject in only_three:
        value = 4146
        value_test = 384
    if subject in both:
        value = 3903
        value_test = 627

    data_train = np.zeros((value,65730)) 
    data_fine = np.zeros((7560,65730)) 
    data_test = np.zeros((value_test,65730))  
    data_fine_test = np.zeros((540,65730))

    glove_train = np.zeros((value,300))   
    glove_fine = np.zeros((7560,300))   
    glove_test = np.zeros((value_test,300))  
    glove_fine_test = np.zeros((540,300))

    numb = 0
    numb_test = 0
    numb_fine_test = 0
    numb_fine = 0

    tot = 0
    tot_fine = 0
    tot_test= 0
    tot_fine_test = 0

    for sub in subjects:

        folder = os.listdir(data_path +sub)
        values = np.zeros((627,212742))     
        values_fine = np.zeros((540,212742))
        numb_fine_tes = 0
        numb_fine1 = 0
        numb_tes = 0
        numb1 = 0
        for fil in folder:
            if sub == subject:
                if fil.startswith('data_180'):
                    Gordon, data = ROI_loader(sub,fil)
                    coord = coltocoord_ROI_ordering(sub,fil)
                    values_fine[numb_fine_tes:numb_fine_tes+data.shape[0],:data.shape[1]] = data
                    if data.shape[0]==180:
                        glove_fine_test[numb_fine_test:numb_fine_test+data.shape[0],:] = vector_180
                    if fil=='data_180concepts_sentences.mat':
                        holder = numb_tes
                    numb_fine_test +=data.shape[0]
                    numb_fine_tes += data.shape[0]
                if fil.startswith('data') and not fil.startswith('data_180'):
                    Gordon, data = ROI_loader(sub,fil)
                    coord = coltocoord_ROI_ordering(sub,fil)
                    values[numb_tes:numb_tes+data.shape[0],:data.shape[1]] = data
                    if data.shape[0]==243:
                        glove_test[numb_test:numb_test+data.shape[0],:] = vector_243
                    if data.shape[0]==384:
                        glove_test[numb_test:numb_test+data.shape[0],:] = vector_384
                    if fil=='data_180concepts_sentences.mat':
                        holder = numb_tes
                    numb_test +=data.shape[0]
                    numb_tes += data.shape[0]
            else:
                if fil.startswith('data_180'):
                    Gordon, data = ROI_loader(sub,fil)
                    coord = coltocoord_ROI_ordering(sub,fil)
                    values_fine[numb_fine1:numb_fine1+data.shape[0],:data.shape[1]] = data
                    if data.shape[0]==180:
                        glove_fine[numb_fine:numb_fine+data.shape[0],:] = vector_180
                    numb_fine +=data.shape[0]
                    numb_fine1+=data.shape[0]
                if fil.startswith('data') and not fil.startswith('data_180'):
                    Gordon, data = ROI_loader(sub,fil)
                    coord = coltocoord_ROI_ordering(sub,fil)
                    values[numb1:numb1+data.shape[0],:data.shape[1]] = data
                    if data.shape[0]==243:
                        glove_train[numb:numb+data.shape[0],:] = vector_243
                    if data.shape[0]==384:
                        glove_train[numb:numb+data.shape[0],:] = vector_384
                    numb +=data.shape[0]
                    numb1+=data.shape[0]
        values = values[~(values==0).all(1)]
        ind_array = 0
        for i in range(333):
            helper = True
            access = 'arr_' +str(i)
            last_dim = last_dim_all[access]
            Roi_coord = np.squeeze(coord[Gordon[i][0]])
            last = np.asarray(np.unique(Roi_coord[:,2], return_counts=True))
            big = np.sum(last_dim[1])
            assert big == sizes[i]
            indexes = Gordon[i][0]
            area = values[:,indexes]
            area_fine = values_fine[:,indexes]
            if area.shape[0]==0:
                helper = False

            if sub == subject:

                if helper:
                    area = np.reshape(area, (values.shape[0],-1))
                    area = matcher(area, last_dim, last, Roi_coord)
                    data_test[tot_test:tot_test+values.shape[0],ind_array:ind_array+area.shape[1]] = area

                area_fine = np.reshape(area_fine, (values_fine.shape[0],-1))
                area_fine = matcher(area_fine, last_dim, last, Roi_coord)
                data_fine_test[tot_fine_test:tot_fine_test+values_fine.shape[0],ind_array:ind_array+area_fine.shape[1]] = area_fine
                ind_array+=sizes[i]
            else :
                if helper:
                    area = np.reshape(area, (values.shape[0],-1))
                    area = matcher(area, last_dim, last, Roi_coord)
                    data_train[tot:(tot+values.shape[0]),ind_array:ind_array+area.shape[1]] = area

                area_fine = np.reshape(area_fine, (values_fine.shape[0],-1))
                area_fine = matcher(area_fine, last_dim, last, Roi_coord)
                data_fine[tot_fine:(tot_fine+values_fine.shape[0]),ind_array:ind_array+area_fine.shape[1]] = area_fine
                ind_array+=sizes[i]
        if sub == subject:
            tot_test+=values.shape[0]
            tot_fine_test+= values_fine.shape[0]
        else:
            tot+=values.shape[0]
            tot_fine+= values_fine.shape[0]

    return data_train, data_test, glove_train, glove_test, data_fine, data_fine_test, glove_fine, glove_fine_test


In [13]:
subject = 'M15' # Selecting the subject

In [14]:
# Load the training and testing data
with open('../file.pkl', 'rb') as file:
    temp_loading_data = pickle.load(file)
data_train, data_test, glove_train, glove_test, data_fine, data_fine_test, glove_fine, glove_fine_test = temp_loading_data

In [15]:
print(data_train.shape)
print(data_test.shape)
print(glove_train.shape)
print(glove_test.shape)
print(data_fine.shape)
print(data_fine_test.shape)
print(glove_fine.shape)
print(glove_fine_test.shape)

print(type(glove_train))

(3903, 65730)
(627, 65730)
(3903, 300)
(627, 300)
(7560, 65730)
(540, 65730)
(7560, 300)
(540, 300)
<class 'numpy.ndarray'>


In [16]:
! nvidia-smi

Mon May  6 19:05:18 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.86                 Driver Version: 551.86         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   49C    P8              3W /   40W |     467MiB /   8188MiB |     30%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

---
## Loading Parameters

In [17]:
value, value_test = class_sizer(subject)
class_fine_test =np.eye(180)
class_fine = np.reshape(np.tile(class_fine_test,(42,1)),(7560,180))
class_fine_test = np.reshape(np.tile(class_fine_test,(3,1)),(540,180)) 
class_train = np.zeros((3903,180))

print(class_fine_test.shape)
print(class_fine.shape)
print(class_train.shape)

(540, 180)
(7560, 180)
(3903, 180)


---
## Loading Data

For other models

In [18]:
# Convert numpy arrays to PyTorch tensors
train_data_tensor = torch.tensor(data_fine)
train_glove_tensor = torch.tensor(glove_fine)
train_class_tensor = torch.tensor(class_fine)

train_data_tensor = train_data_tensor.to(torch.float32)
train_glove_tensor = train_glove_tensor.to(torch.float32)
train_class_tensor = train_class_tensor.to(torch.float32)

test_data_tensor = torch.tensor(data_fine_test)
test_glove_tensor = torch.tensor(glove_fine_test)
test_class_tensor = torch.tensor(class_fine_test)

test_data_tensor = test_data_tensor.to(torch.float32)
test_glove_tensor = test_glove_tensor.to(torch.float32)
test_class_tensor = test_class_tensor.to(torch.float32)

train_data_tensor = train_data_tensor.to(device)
train_glove_tensor = train_glove_tensor.to(device)
train_class_tensor = train_class_tensor.to(device)

test_data_tensor = test_data_tensor.to(device)
test_glove_tensor = test_glove_tensor.to(device)
test_class_tensor = test_class_tensor.to(device)

In [19]:
# reg_train_data_tensor = np.concatenate((data_fine, data_train), axis=0)
# reg_train_glove_tensor = np.concatenate((glove_fine, glove_train), axis=0)
# reg_train_class_tensor = np.concatenate((class_fine, class_train), axis=0)

# reg_test_data_tensor = np.concatenate((data_fine_test, data_test), axis=0)
# reg_test_glove_tensor = np.concatenate((glove_fine_test, glove_test), axis=0)
# reg_test_class_tensor = np.concatenate((class_fine_test, class_fine_test), axis=0)

# reg_train_data_tensor = torch.tensor(reg_train_data_tensor)
# reg_train_glove_tensor = torch.tensor(reg_train_glove_tensor)
# reg_train_class_tensor = torch.tensor(reg_train_class_tensor)

# reg_train_data_tensor = reg_train_data_tensor.to(torch.float32)
# reg_train_glove_tensor = reg_train_glove_tensor.to(torch.float32)
# reg_train_class_tensor = reg_train_class_tensor.to(torch.float32)

# reg_test_data_tensor = torch.tensor(reg_test_data_tensor)
# reg_test_glove_tensor = torch.tensor(reg_test_glove_tensor)
# reg_test_class_tensor = torch.tensor(reg_test_class_tensor)

# reg_test_data_tensor = reg_test_data_tensor.to(torch.float32)
# reg_test_glove_tensor = reg_test_glove_tensor.to(torch.float32)
# reg_test_class_tensor = reg_test_class_tensor.to(torch.float32)

# reg_train_data_tensor = reg_train_data_tensor.to(device)
# reg_train_glove_tensor = reg_train_glove_tensor.to(device) 
# reg_train_class_tensor = reg_train_class_tensor.to(device)

# reg_test_data_tensor = reg_test_data_tensor.to(device)   
# reg_test_glove_tensor = reg_test_glove_tensor.to(device)
# reg_test_class_tensor = reg_test_class_tensor.to(device)

For Mean Regularization

In [20]:
# Convert numpy arrays to PyTorch tensors
mean_train_data_tensor = torch.tensor(data_train)
mean_train_glove_tensor = torch.tensor(glove_train)
mean_train_class_tensor = torch.tensor(class_train)

mean_train_data_tensor = mean_train_data_tensor.to(torch.float32)
mean_train_glove_tensor = mean_train_glove_tensor.to(torch.float32)
mean_train_class_tensor = mean_train_class_tensor.to(torch.float32)

mean_test_data_tensor = torch.tensor(data_fine_test)
mean_test_glove_tensor = torch.tensor(glove_fine_test)
mean_test_class_tensor = torch.tensor(class_fine_test)

mean_test_data_tensor = mean_test_data_tensor.to(torch.float32)
mean_test_glove_tensor = mean_test_glove_tensor.to(torch.float32)
mean_test_class_tensor = mean_test_class_tensor.to(torch.float32)

mean_train_data_tensor = mean_train_data_tensor.to(device)
mean_train_glove_tensor = mean_train_glove_tensor.to(device)
mean_train_class_tensor = mean_train_class_tensor.to(device)

mean_test_data_tensor = mean_test_data_tensor.to(device)
mean_test_glove_tensor = mean_test_glove_tensor.to(device)
mean_test_class_tensor = mean_test_class_tensor.to(device)

----
## Base model

In [21]:
class EncDecSmallModel(nn.Module):
    def __init__(self, rate=0.4, glove_size=300, dense_size=2000, fMRI_size=65730, gordon_areas=333, class_size=180):
        super(EncDecSmallModel, self).__init__()
        
        self.dense_first = nn.Sequential(nn.Linear(fMRI_size, dense_size),nn.LeakyReLU(0.3),nn.BatchNorm1d(dense_size),nn.Dropout(rate))
        self.out_glove = nn.Linear(dense_size, glove_size)
        self.out_class = nn.Sequential(
            nn.Linear(glove_size, class_size),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.dense_first(x)
        out_glove = self.out_glove(x)
        out_class = self.out_class(out_glove)
        return out_glove, out_class

---
### Training the base model

In [22]:
# Create DataLoader
batch_size = 128
train_dataset = TensorDataset(train_data_tensor, train_glove_tensor, train_class_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(test_data_tensor, test_glove_tensor, test_class_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Instantiate the model
model = EncDecSmallModel().to(device)

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define loss function
loss_fn_glove = MeanDistanceLoss(CosineProximityLoss())
loss_fn_class = nn.CrossEntropyLoss()

criterion_glove = pairwise_evaluation
criterion_class = top_5 

# Define number of epochs
epochs = 15

# Define learning rate scheduler
scheduler = StepLR(optimizer, step_size=5, gamma=0.25)

# Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    running_loss_glove = 0.0
    running_loss_class = 0.0
    cnt2 = 0
    for batch_data, batch_glove, batch_class in tqdm.tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}'):
        optimizer.zero_grad()
        output_glove, output_class = model(batch_data)
        loss_glove = loss_fn_glove(output_glove, batch_glove)
        loss_class = loss_fn_class(output_class, batch_class)
        loss = loss_class + loss_glove
        loss.backward()
        loss.dtype
        optimizer.step()
        
        running_loss_glove += loss_glove.item() * batch_data.size(0)
        running_loss_class += loss_class.item() * batch_data.size(0)
        cnt2+=1
    
    epoch_loss_glove = running_loss_glove / len(train_loader.dataset)
    epoch_loss_class = running_loss_class / len(train_loader.dataset)
    total_a1 = 0
    total_a5 = 0
    total_a10 = 0
    cnt = 0
    pair = 0
    
    with torch.no_grad():
        for inputs, targets_glove, targets_class in test_loader:
            outputs_glove, outputs_class = model(inputs)
            _, predicted_class = torch.max(outputs_class, 1)
            loss_glove = criterion_glove(outputs_glove,targets_glove)

            # Calculate average accuracy
            a1, a5, a10 = top_5(outputs_class.cpu().numpy(), targets_class.cpu().numpy())
            total_a1 += a1
            total_a5 += a5
            total_a10 += a10
            cnt+=1
            pair += loss_glove

    average_a1 = total_a1 / cnt
    average_a5 = total_a5 / cnt
    average_a10 = total_a10 / cnt
    pair = pair /cnt
    running_loss = epoch_loss_class + epoch_loss_glove
    average_loss = running_loss / cnt2
    print(f'Epoch [{epoch+1}/{epochs}], Average Loss: {average_loss:.4f}, Average Top 1 Accuracy: {average_a1:.4f}, Average Top 5 Accuracy: {average_a5:.4f}, Average Top 10 Accuracy: {average_a10:.4f}, Pairwise: {pair:.4f}')
    scheduler.step()

Epoch 1/15: 100%|██████████| 60/60 [00:58<00:00,  1.02it/s]


Epoch [1/15], Average Loss: 0.0864, Average Top 1 Accuracy: 0.0109, Average Top 5 Accuracy: 0.0453, Average Top 10 Accuracy: 0.0766, Pairwise: 0.7173


Epoch 2/15:   8%|▊         | 5/60 [00:04<00:54,  1.01it/s]


KeyboardInterrupt: 

---
## Big Model

In [23]:
class EncDecBigModel(nn.Module):
    def __init__(self, concating=True):
        super(EncDecBigModel, self).__init__()
        # Parameters
        self.rate = 0.4
        self.glove_size = 300
        self.fMRI_size = 65730
        self.gordon_areas = 333
        self.class_size = 180

        # Load sizes
        sizes = np.load('../data/look_ups/sizes.npy')
        reduced = np.load('../data/look_ups/reduced_sizes.npy')

        # Small ROI dense layers
        self.dense_layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()
        index = 0
        for i in range(self.gordon_areas):
            new_index = index + sizes[i]
            self.dense_layers.append(nn.Linear(sizes[i], reduced[i]))
            self.batch_norms.append(nn.BatchNorm1d(reduced[i], track_running_stats=True))
            index = new_index

        # Dropout layer
        self.dropout = nn.Dropout(self.rate)

        # Glove layer
        self.dense_glove = nn.Linear(sum(reduced), self.glove_size)

        # Classification layer
        self.dense_class = nn.Linear(self.glove_size, self.class_size)

        self.concat = concating

    def forward(self, x):
        branch_outputs = []
        index = 0
        for i, (layer, bn) in enumerate(zip(self.dense_layers, self.batch_norms)):
            small_input = x[:, index:index + layer.in_features]
            small_out = F.leaky_relu(layer(small_input), negative_slope=0.3)
            small_out = bn(small_out)
            branch_outputs.append(small_out)
            index += layer.in_features
        
        con = torch.cat(branch_outputs, dim=1)
        con = self.dropout(con)

        # Glove layer
        out_glove = self.dense_glove(con)

        # Classification layer
        out_class = F.softmax(self.dense_class(out_glove), dim=1)

        if self.concat:
            return out_glove, out_class, con
        else:
            return out_glove, out_class

---
### Training the big model

In [24]:
# Create DataLoader
batch_size = 180
train_dataset = TensorDataset(train_data_tensor, train_glove_tensor, train_class_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(test_data_tensor, test_glove_tensor, test_class_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Instantiate the model
model = EncDecBigModel(concating=False).to(device)

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define loss function
loss_fn_glove = MeanDistanceLoss(CosineProximityLoss())
loss_fn_class = nn.CrossEntropyLoss()

criterion_glove = pairwise_evaluation
criterion_class = nn.CrossEntropyLoss()  

# Define number of epochs
epochs = 20

# Define learning rate scheduler
scheduler = StepLR(optimizer, step_size=5, gamma=0.25)

# Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    running_loss_glove = 0.0
    running_loss_class = 0.0
    cnt2 = 0
    for batch_data, batch_glove, batch_class in tqdm.tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}'):
        optimizer.zero_grad()
        output_glove, output_class = model(batch_data)
        loss_glove = loss_fn_glove(output_glove, batch_glove)
        loss_class = loss_fn_class(output_class, batch_class)
        loss = loss_class + loss_glove
        loss.backward()
        loss.dtype
        optimizer.step()
        
        running_loss_glove += loss_glove.item() * batch_data.size(0)
        running_loss_class += loss_class.item() * batch_data.size(0)
        cnt2+=1
    
    epoch_loss_glove = running_loss_glove / len(train_loader.dataset)
    epoch_loss_class = running_loss_class / len(train_loader.dataset)
    total_a1 = 0
    total_a5 = 0
    total_a10 = 0
    cnt = 0
    pair = 0
    
    with torch.no_grad():
        for inputs, targets_glove, targets_class in test_loader:
            outputs_glove, outputs_class = model(inputs)
            _, predicted_class = torch.max(outputs_class, 1)
            loss_glove = criterion_glove(outputs_glove,targets_glove)

            # Calculate average accuracy
            a1, a5, a10 = top_5(outputs_class.cpu().numpy(), targets_class.cpu().numpy())
            total_a1 += a1
            total_a5 += a5
            total_a10 += a10
            cnt+=1
            pair += loss_glove


    average_a1 = total_a1 / cnt
    average_a5 = total_a5 / cnt
    average_a10 = total_a10 / cnt
    pair = pair /cnt
    running_loss = epoch_loss_class + epoch_loss_glove
    average_loss = running_loss / cnt2
    print(f'Epoch [{epoch+1}/{epochs}], Average Loss: {average_loss:.4f}, Average Top 1 Accuracy: {average_a1:.4f}, Average Top 5 Accuracy: {average_a5:.4f}, Average Top 10 Accuracy: {average_a10:.4f}, pairwise: {pair:.4f}')
    scheduler.step()

Epoch 1/20: 100%|██████████| 42/42 [00:25<00:00,  1.64it/s]


Epoch [1/20], Average Loss: 0.1232, Average Top 1 Accuracy: 0.0130, Average Top 5 Accuracy: 0.0667, Average Top 10 Accuracy: 0.1352, pairwise: 0.7930


Epoch 2/20:   2%|▏         | 1/42 [00:01<00:49,  1.21s/it]


KeyboardInterrupt: 

---
## Autoencoder Model

In [21]:
class Autoencoder(nn.Module):
    def __init__(self, mean = False, training=True):
        super(Autoencoder, self).__init__()

        rate = 0.0001
        dense_size = 200
        glove_size = 300
        fMRI_size = 65730
        reduced_size = 3221
        gordon_areas = 333

        sizes = np.load('../data/look_ups/sizes.npy')
        reduced = np.load('../data/look_ups/reduced_sizes.npy')

        self.dense_layers = nn.ModuleList([nn.Linear(sizes[i], reduced[i]) for i in range(len(sizes))])
        self.batch_norm = nn.ModuleList([nn.BatchNorm1d(reduced[i]) for i in range(len(sizes))])
        self.dropout = nn.Dropout(rate)

        self.batch_norm11 = nn.BatchNorm1d(reduced_size)
        self.dropout11 = nn.Dropout(rate)

        self.dense1 = nn.Linear(reduced_size, dense_size)
        self.leaky_relu = nn.LeakyReLU(0.3)
        self.batch_norm1 = nn.BatchNorm1d(dense_size)
        self.dropout1 = nn.Dropout(rate)

        self.dense2 = nn.Linear(dense_size, glove_size)
        self.batch_norm2 = nn.BatchNorm1d(glove_size)
        self.dropout2 = nn.Dropout(rate)
        if(training==False):
            self.dense2.weight.requires_grad = False  # Freeze the parameters
            self.dense2.bias.requires_grad = False
        


        self.dense3 = nn.Linear(glove_size, 180)
        self.softmax = nn.Softmax(dim=1)

        if(training==False):
            self.dense3.weight.requires_grad = False  # Freeze the parameters
            self.dense3.bias.requires_grad = False

        self.dense4 = nn.Linear(dense_size, reduced_size)
        self.batch_norm3 = nn.BatchNorm1d(reduced_size)
        self.dropout3 = nn.Dropout(rate)

        self.dense_layers_transpose = nn.ModuleList([nn.Linear(reduced[i], sizes[i]) for i in range(len(sizes))])
        self.batch_norm4 = nn.ModuleList([nn.BatchNorm1d(size) for size in sizes])

        self.mean = mean

    def forward(self, x):
        branch_outputs = []
        index = 0
        gordon_areas = 333
        sizes = np.load('../data/look_ups/sizes.npy')
        reduced = np.load('../data/look_ups/reduced_sizes.npy')
        reduced_size = 3221
        for i in range(gordon_areas):
            new_index = index + sizes[i]
            small_input = x[:, index:new_index]
            dense_out = self.leaky_relu(self.dense_layers[i](small_input))
            dense_out = self.batch_norm[i](dense_out)
            dense_out = self.dropout(dense_out)
            branch_outputs.append(dense_out)
            index = new_index

        concat = torch.cat(branch_outputs, dim=1)
        dense1_out = self.batch_norm11(concat)
        dense1_out = self.dropout11(dense1_out)

        out_further = self.leaky_relu(self.dense1(dense1_out))
        out_further = self.batch_norm1(out_further)
        out_further = self.dropout1(out_further)

        out_glove = self.leaky_relu(self.dense2(out_further))
        out_glove = self.batch_norm2(out_glove)
        out_glove = self.dropout2(out_glove)

        out_class = self.softmax(self.dense3(out_glove))

        dense4_out = self.leaky_relu(self.dense4(out_further))
        dense4_out = self.batch_norm3(dense4_out)
        dense4_out = self.dropout3(dense4_out)

        branch_outputs1 = []
        index1 = 0
        for j in range(gordon_areas):
            new_index1 = index1 + reduced[j]
            small_input = dense4_out[:, index1:new_index1]

            dense_out = self.leaky_relu(self.dense_layers_transpose[j](small_input))
            dense_out = self.batch_norm4[j](dense_out)
            branch_outputs1.append(dense_out)
            index1 = new_index1
        out = torch.cat(branch_outputs1, dim=1)
        
        if not self.mean:
            return out, out_glove, out_class
        else:
            return out, out_glove, out_class, concat, dense1_out

---
### Training Autoencoder Model

In [29]:
# Create DataLoader
batch_size = 180
train_dataset = TensorDataset(train_data_tensor, train_glove_tensor, train_class_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(test_data_tensor, test_glove_tensor, test_class_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

model = Autoencoder().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

loss_fn_glove = MeanDistanceLoss(CosineProximityLoss())
loss_fn_class = nn.CrossEntropyLoss()
criterion_glove = pairwise_evaluation
criterion_class = nn.CrossEntropyLoss()
loss_fn_autoencoder = CosineProximityLoss()

epochs = 20
scheduler = StepLR(optimizer, step_size=5, gamma=0.25)

In [30]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    running_loss_glove = 0.0
    running_loss_class = 0.0
    running_loss_autoencoder = 0.0
    cnt2 = 0
    for batch_data, batch_glove, batch_class in tqdm.tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}'):

        optimizer.zero_grad()
        out, output_glove, output_class = model(batch_data)
        loss_glove = loss_fn_glove(output_glove, batch_glove)
        loss_class = loss_fn_class(output_class, batch_class)
        loss_autoencoder = loss_fn_autoencoder(batch_data, out)
        loss = loss_class + loss_autoencoder + loss_glove
        loss.backward()
        loss.dtype
        optimizer.step()
        
        running_loss_glove += loss_glove.item() * batch_data.size(0)
        running_loss_class += loss_class.item() * batch_data.size(0)
        running_loss_autoencoder += loss_autoencoder.item() * batch_data.size(0)
        cnt2+=1
    
    epoch_loss_glove = running_loss_glove / len(train_loader.dataset)
    epoch_loss_class = running_loss_class / len(train_loader.dataset)
    epoch_loss_autoencoder = running_loss_autoencoder / len(train_loader.dataset)
    total_a1 = 0
    total_a5 = 0
    total_a10 = 0
    cnt = 0
    pair = 0

    with torch.no_grad():
        for inputs, targets_glove, targets_class in test_loader:
            out, outputs_glove, outputs_class = model(inputs)
            _, predicted_class = torch.max(outputs_class, 1)
            loss_glove = criterion_glove(outputs_glove, targets_glove)

            # Calculate average accuracy
            a1, a5, a10 = top_5(outputs_class.cpu().numpy(), targets_class.cpu().numpy())
            total_a1 += a1
            total_a5 += a5
            total_a10 += a10
            pair += loss_glove
            cnt+=1

    average_a1 = total_a1 / cnt
    average_a5 = total_a5 / cnt
    average_a10 = total_a10 / cnt
    running_loss = epoch_loss_class + epoch_loss_glove + epoch_loss_autoencoder
    average_loss = running_loss / cnt2
    pair = pair /cnt
    print(f'Epoch [{epoch+1}/{epochs}], Average Loss: {average_loss:.4f}, Average Top 1 Accuracy: {average_a1:.4f}, Average Top 5 Accuracy: {average_a5:.4f}, Average Top 10 Accuracy: {average_a10:.4f}, Pairwise: {pair:.4f}')
    
    scheduler.step()

Epoch 1/20: 100%|██████████| 42/42 [01:07<00:00,  1.60s/it]


Epoch [1/20], Average Loss: 0.1426, Average Top 1 Accuracy: 0.0167, Average Top 5 Accuracy: 0.0685, Average Top 10 Accuracy: 0.1130, Pairwise: 0.7655


Epoch 2/20:   2%|▏         | 1/42 [00:03<02:17,  3.36s/it]


KeyboardInterrupt: 

# Training the Mean Regularizarion Model

PRE

In [28]:
# Create DataLoader
batch_size = 180
mean_train_dataset = TensorDataset(mean_train_data_tensor, mean_train_glove_tensor, mean_train_class_tensor)
mean_train_loader = DataLoader(mean_train_dataset, batch_size=batch_size, shuffle=True)

mean_test_dataset = TensorDataset(mean_test_data_tensor, mean_test_glove_tensor, mean_test_class_tensor)
mean_test_loader = DataLoader(mean_test_dataset, batch_size=batch_size)

model = Autoencoder(training=False).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

loss_fn_glove = MeanDistanceLoss(CosineProximityLoss())
loss_fn_class = nn.CrossEntropyLoss()
criterion_glove = pairwise_evaluation
criterion_class = nn.CrossEntropyLoss()
loss_fn_autoencoder = CosineProximityLoss()

epochs = 20
scheduler = StepLR(optimizer, step_size=5, gamma=0.25)

In [29]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    running_loss_glove = 0.0
    running_loss_class = 0.0
    running_loss_autoencoder = 0.0
    cnt2 = 0
    for batch_data, batch_glove, batch_class in tqdm.tqdm(mean_train_loader, desc=f'Epoch {epoch+1}/{epochs}'):

        optimizer.zero_grad()
        out, output_glove, output_class = model(batch_data)
        loss_glove = loss_fn_glove(output_glove, batch_glove)
        loss_class = loss_fn_class(output_class, batch_class)
        loss_autoencoder = loss_fn_autoencoder(batch_data, out)
        loss = 0*loss_class + loss_autoencoder + 0*loss_glove
        loss.backward()
        loss.dtype
        optimizer.step()
        
        running_loss_glove += loss_glove.item() * batch_data.size(0)
        running_loss_class += loss_class.item() * batch_data.size(0)
        running_loss_autoencoder += loss_autoencoder.item() * batch_data.size(0)
        cnt2+=1
    
    epoch_loss_glove = running_loss_glove / len(mean_train_loader.dataset)
    epoch_loss_class = running_loss_class / len(mean_train_loader.dataset)
    epoch_loss_autoencoder = running_loss_autoencoder / len(mean_train_loader.dataset)

    auto_loss_sum = 0
    cnt = 0
    
    with torch.no_grad():
        for inputs, targets_glove, targets_class in mean_test_loader:
            out, outputs_glove, outputs_class = model(inputs)
            auto_loss = loss_fn_autoencoder(inputs, out)

            auto_loss_sum += auto_loss
            cnt+=1

    # Calculate average accuracy
    average_a1 = total_a1 / cnt
    average_a5 = total_a5 / cnt
    average_a10 = total_a10 / cnt
    running_loss = epoch_loss_class + epoch_loss_glove + epoch_loss_autoencoder
    average_loss = running_loss / cnt2
    print(f'Epoch [{epoch+1}/{epochs}], Auto Loss: {auto_loss_sum:.4f}, )')
    scheduler.step()

Epoch 1/20: 100%|██████████| 22/22 [00:16<00:00,  1.30it/s]


Epoch [1/20], Auto Loss: 2.6741, )


Epoch 2/20: 100%|██████████| 22/22 [00:18<00:00,  1.21it/s]


Epoch [2/20], Auto Loss: 2.4644, )


Epoch 3/20: 100%|██████████| 22/22 [00:17<00:00,  1.25it/s]


...

Epoch [18/20], Auto Loss: 2.0209, )


Epoch 19/20: 100%|██████████| 22/22 [00:15<00:00,  1.40it/s]


Epoch [19/20], Auto Loss: 2.0201, )


Epoch 20/20: 100%|██████████| 22/22 [00:17<00:00,  1.25it/s]


Epoch [20/20], Auto Loss: 2.0191, )


In [30]:
# save weights
torch.save(model.state_dict(), 'pre_autoencoder1.pth')

Auto

In [31]:
# Create DataLoader
batch_size = 180
train_dataset = TensorDataset(train_data_tensor, train_glove_tensor, train_class_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(test_data_tensor, test_glove_tensor, test_class_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

model = Autoencoder(training=True, mean=False).to(device)
# load weights
model.load_state_dict(torch.load('../pre_autoencoder1.pth'))
optimizer = optim.Adam(model.parameters(), lr=0.001)

loss_fn_glove = MeanDistanceLoss(CosineProximityLoss())
loss_fn_class = nn.CrossEntropyLoss()
criterion_glove = pairwise_evaluation
criterion_class = nn.CrossEntropyLoss()
loss_fn_autoencoder = CosineProximityLoss()

epochs = 20
scheduler = StepLR(optimizer, step_size=5, gamma=0.25)

In [32]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    running_loss_glove = 0.0
    running_loss_class = 0.0
    running_loss_autoencoder = 0.0
    cnt2 = 0
    for batch_data, batch_glove, batch_class in tqdm.tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}'):

        optimizer.zero_grad()
        out, output_glove, output_class = model(batch_data)
        loss_glove = loss_fn_glove(output_glove, batch_glove)
        loss_class = loss_fn_class(output_class, batch_class)
        loss_autoencoder = loss_fn_autoencoder(batch_data, out)
        loss = loss_class + loss_autoencoder + loss_glove
        loss.backward()
        loss.dtype
        optimizer.step()
        
        running_loss_glove += loss_glove.item() * batch_data.size(0)
        running_loss_class += loss_class.item() * batch_data.size(0)
        running_loss_autoencoder += loss_autoencoder.item() * batch_data.size(0)
        cnt2+=1
    
    epoch_loss_glove = running_loss_glove / len(train_loader.dataset)
    epoch_loss_class = running_loss_class / len(train_loader.dataset)
    epoch_loss_autoencoder = running_loss_autoencoder / len(train_loader.dataset)
    total_a1 = 0
    total_a5 = 0
    total_a10 = 0
    cnt = 0
    
    with torch.no_grad():
        for inputs, targets_glove, targets_class in test_loader:
            out, outputs_glove, outputs_class = model(inputs)
            _, predicted_class = torch.max(outputs_class, 1)

            a1, a5, a10 = top_5(outputs_class.cpu().numpy(), targets_class.cpu().numpy())
            total_a1 += a1
            total_a5 += a5
            total_a10 += a10
            cnt+=1

    # Calculate average accuracy
    average_a1 = total_a1 / cnt
    average_a5 = total_a5 / cnt
    average_a10 = total_a10 / cnt
    running_loss = epoch_loss_class + epoch_loss_glove + epoch_loss_autoencoder
    average_loss = running_loss / cnt2
    print(f'Epoch [{epoch+1}/{epochs}], Average Loss: {average_loss:.4f}, Average Top 1 Accuracy: {average_a1:.4f}, Average Top 5 Accuracy: {average_a5:.4f}, Average Top 10 Accuracy: {average_a10:.4f}')
    
    scheduler.step()

Epoch 1/20: 100%|██████████| 42/42 [00:31<00:00,  1.33it/s]


Epoch [1/20], Average Loss: 0.1332, Average Top 1 Accuracy: 0.0296, Average Top 5 Accuracy: 0.0685, Average Top 10 Accuracy: 0.1167


Epoch 2/20: 100%|██████████| 42/42 [00:32<00:00,  1.31it/s]


Epoch [2/20], Average Loss: 0.1302, Average Top 1 Accuracy: 0.0352, Average Top 5 Accuracy: 0.1019, Average Top 10 Accuracy: 0.1870


Epoch 3/20: 100%|██████████| 42/42 [00:34<00:00,  1.22it/s]


...

Epoch [18/20], Average Loss: 0.1110, Average Top 1 Accuracy: 0.0519, Average Top 5 Accuracy: 0.1722, Average Top 10 Accuracy: 0.2519


Epoch 19/20: 100%|██████████| 42/42 [00:29<00:00,  1.40it/s]


Epoch [19/20], Average Loss: 0.1109, Average Top 1 Accuracy: 0.0574, Average Top 5 Accuracy: 0.1759, Average Top 10 Accuracy: 0.2519


Epoch 20/20: 100%|██████████| 42/42 [00:29<00:00,  1.41it/s]


Epoch [20/20], Average Loss: 0.1107, Average Top 1 Accuracy: 0.0537, Average Top 5 Accuracy: 0.1722, Average Top 10 Accuracy: 0.2519


In [33]:
# save weights
torch.save(model.state_dict(), 'autoencoder1.pth')

Mean Model

In [22]:
def mean_Concat(Concat_pred, Concat_mean, first):
    out = torch.zeros((180, Concat_pred.shape[1]), dtype=Concat_pred.dtype, device=Concat_pred.device)
    for i in range(180):
        values = Concat_pred[i::180, :]  # Extracting values from Concat_pred
        out[i, :] = torch.mean(values, dim=0)  # Compute mean along dim 0
    if first:
        return out
    else:
        return (out + Concat_mean) / 2

In [23]:
def get_tensors(mean, mean_den):
    # Calculate the number of repetitions based on the desired shapes
    mean_tot_reps = (42, 1)
    mean_test_reps = (3, 1)

    # Tile and reshape mean tensor
    mean_tot = mean.repeat(mean_tot_reps[0], mean_tot_reps[1]).view(-1, mean.shape[1])

    # Tile and reshape mean_den tensor
    mean_den_tot = mean_den.repeat(mean_tot_reps[0], mean_tot_reps[1]).view(-1, mean_den.shape[1])

    # Tile and reshape mean tensor for test
    mean_test = mean.repeat(mean_test_reps[0], mean_test_reps[1]).view(-1, mean.shape[1])

    # Tile and reshape mean_den tensor for test
    mean_den_test = mean_den.repeat(mean_test_reps[0], mean_test_reps[1]).view(-1, mean_den.shape[1])

    return mean_tot, mean_den_tot, mean_test, mean_den_test



In [24]:
data_fine_tensor = torch.tensor(data_fine)
data_fine_tensor = data_fine_tensor.to(torch.float32)
data_fine_tensor = data_fine_tensor.to(device)

In [25]:
# Function to make dataloaders for mean regularization 
def get_loaders(batch_size, data_train, glove_train, class_train, data_test, glove_test, class_fine_test, mean_tot, mean_den_tot, mean_test, mean_den_test):
    # Convert data to tensors
    data_train_tensor = torch.tensor(data_train).to(torch.float32).to(device)
    class_train_tensor = torch.tensor(class_train).to(torch.float32).to(device)
    glove_train_tensor = torch.tensor(glove_train).to(torch.float32).to(device)
    mean_tot_tensor = torch.tensor(mean_tot).to(torch.float32).to(device)
    mean_den_tot_tensor = torch.tensor(mean_den_tot).to(torch.float32).to(device)
    data_test_tensor = torch.tensor(data_test).to(torch.float32).to(device)
    class_fine_test_tensor = torch.tensor(class_fine_test).to(torch.float32).to(device)
    glove_test_tensor = torch.tensor(glove_test).to(torch.float32).to(device)
    mean_test_tensor = torch.tensor(mean_test).to(torch.float32).to(device)
    mean_den_test_tensor = torch.tensor(mean_den_test).to(torch.float32).to(device)

    # Create TensorDatasets
    train_dataset = TensorDataset(data_train_tensor, class_train_tensor, glove_train_tensor, mean_tot_tensor, mean_den_tot_tensor)
    test_dataset = TensorDataset(data_test_tensor, class_fine_test_tensor, glove_test_tensor, mean_test_tensor, mean_den_test_tensor)

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader


In [26]:

model = Autoencoder(training=True, mean=True).to(device)
# load weights
model.load_state_dict(torch.load('../autoencoder1.pth'))
optimizer = optim.Adam(model.parameters(), lr=0.001)

loss_fn_glove = MeanDistanceLoss(CosineProximityLoss())
loss_fn_class = nn.CrossEntropyLoss()
loss_fn_autoencoder = CosineProximityLoss()
loss_fn_concat = MeanDistanceLoss(CosineProximityLoss())
loss_fn_dense = MeanDistanceLoss(CosineProximityLoss())
criterion_glove = pairwise_evaluation

epochs = 20
scheduler = StepLR(optimizer, step_size=5, gamma=0.25)

In [27]:
model.eval()

# Perform inference
with torch.no_grad():
    rec, glove, clas, pred, pred_den = model(data_fine_tensor)


In [28]:
mean = mean_Concat(pred, torch.zeros((180, pred.shape[1]), dtype=torch.float32), True)
mean_den = mean_Concat(pred_den, torch.zeros((180,pred_den.shape[1]),dtype=torch.float32),True)

In [29]:
# function to train the model
def train_model(model, optimizer, loss_fn_glove, loss_fn_class, loss_fn_autoencoder, loss_concat, loss_dense, train_loader, test_loader, epochs, scheduler, mean, mean_den):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        running_loss_glove = 0.0
        running_loss_class = 0.0
        running_loss_autoencoder = 0.0
        running_loss_concat = 0.0
        running_loss_dense = 0.0
        cnt2 = 0
        for batch_data, batch_class, batch_glove, batch_mean, batch_mean_den in tqdm.tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}'):
            optimizer.zero_grad()
            rec, glove, clas, pred, pred_den = model(batch_data)
            loss_glove = loss_fn_glove(glove, batch_glove)
            loss_class = loss_fn_class(clas, batch_class)
            loss_autoencoder = loss_fn_autoencoder(batch_data, rec)
            loss_concat = loss_fn_concat(pred, batch_mean)
            loss_dense = loss_fn_dense(pred_den, batch_mean_den)
            loss = loss_class + loss_autoencoder + loss_glove + 2.2*loss_concat + 2.0*loss_dense
            loss.backward()
            loss.dtype
            optimizer.step()
            running_loss_glove += loss_glove.item() * batch_data.size(0)
            running_loss_class += loss_class.item() * batch_data.size(0)
            running_loss_autoencoder += loss_autoencoder.item() * batch_data.size(0)
            running_loss_concat += loss_concat.item() * batch_data.size(0)
            running_loss_dense += loss_dense.item() * batch_data.size(0)
            cnt2+=1
        epoch_loss_glove = running_loss_glove / len(train_loader.dataset)
        epoch_loss_class = running_loss_class / len(train_loader.dataset)
        epoch_loss_autoencoder = running_loss_autoencoder / len(train_loader.dataset)
        total_a1 = 0
        total_a5 = 0
        total_a10 = 0
        cnt = 0
        pair = 0
        with torch.no_grad():
            for inputs, targets_class, targets_glove, mean, mean_den in test_loader:
                rec, glove, clas, pred, pred_den = model(inputs)
                loss_glove = criterion_glove(glove,targets_glove)
                loss_class = loss_fn_class(clas, targets_class)
                loss_autoencoder = loss_fn_autoencoder(inputs, rec)
                loss_concat = loss_fn_concat(pred, mean)
                loss_dense = loss_fn_dense(pred_den, mean_den)
                
                # Calculate average accuracy
                a1, a5, a10 = top_5(clas.cpu().numpy(), targets_class.cpu().numpy())
                total_a1 += a1
                total_a5 += a5
                total_a10 += a10
                pair += loss_glove
                cnt+=1
        
        pair = pair /cnt
        average_a1 = total_a1 / cnt
        average_a5 = total_a5 / cnt
        average_a10 = total_a10 / cnt
        running_loss = epoch_loss_class + epoch_loss_glove + epoch_loss_autoencoder
        average_loss = running_loss / cnt2
        print(f'Epoch [{epoch+1}/{epochs}], Average Loss: {average_loss:.4f}, Average Top 1 Accuracy: {average_a1:.4f}, Average Top 5 Accuracy: {average_a5:.4f}, Average Top 10 Accuracy: {average_a10:.4f}, Pairwise: {pair:.4f}')
        scheduler.step()


In [30]:
epochs = 5

In [31]:
for i in range(4):
    mean_tot, mean_den_tot, mean_test, mean_den_test = get_tensors(mean, mean_den)
    rec, glove, clas, pred, pred_den = model(data_fine_tensor)
    train_loader, test_loader = get_loaders(128, data_fine, glove_fine, class_fine, data_fine_test, glove_fine_test, class_fine_test, mean_tot, mean_den_tot, mean_test, mean_den_test)
    train_model(model, optimizer, loss_fn_glove, loss_fn_class, loss_fn_autoencoder, loss_fn_concat, loss_fn_dense, train_loader, test_loader, epochs, scheduler, mean, mean_den)
    model.eval()
    # Perform inference
    with torch.no_grad():
        rec, glove, clas, pred, pred_den = model(data_fine_tensor)
    mean = mean_Concat(pred, mean, False)
    mean_den = mean_Concat(pred_den, mean_den, False)

C:\Users\tanve\AppData\Local\Temp\ipykernel_37644\2506654132.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mean_tot_tensor = torch.tensor(mean_tot).to(torch.float32).to(device)
C:\Users\tanve\AppData\Local\Temp\ipykernel_37644\2506654132.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mean_den_tot_tensor = torch.tensor(mean_den_tot).to(torch.float32).to(device)
C:\Users\tanve\AppData\Local\Temp\ipykernel_37644\2506654132.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mean_test_tensor = torch.tensor(mean_test).to(torch.floa

Epoch [1/5], Average Loss: 0.0809, Average Top 1 Accuracy: 0.0643, Average Top 5 Accuracy: 0.1708, Average Top 10 Accuracy: 0.2469, Pairwise: 0.8152


Epoch 2/5: 100%|██████████| 60/60 [02:14<00:00,  2.25s/it]


Epoch [2/5], Average Loss: 0.0793, Average Top 1 Accuracy: 0.0690, Average Top 5 Accuracy: 0.1583, Average Top 10 Accuracy: 0.2397, Pairwise: 0.8086


Epoch 3/5: 100%|██████████| 60/60 [02:14<00:00,  2.25s/it]


...

Epoch [5/5], Average Loss: 0.0759, Average Top 1 Accuracy: 0.0509, Average Top 5 Accuracy: 0.1511, Average Top 10 Accuracy: 0.2422, Pairwise: 0.7858


Epoch 1/5: 100%|██████████| 60/60 [03:16<00:00,  3.27s/it]


Epoch [1/5], Average Loss: 0.0736, Average Top 1 Accuracy: 0.0627, Average Top 5 Accuracy: 0.1879, Average Top 10 Accuracy: 0.2679, Pairwise: 0.7890


Epoch 2/5:   5%|▌         | 3/60 [00:11<03:30,  3.69s/it]


KeyboardInterrupt: 